In [1]:
# General modules
import sys
import os
import session_info
import warnings
from pyprojroot.here import here
import glob
import pandas as pd
import numpy as np
import pickle

# Setting some parameters
warnings.filterwarnings("ignore")
sys.path.insert(1, str(here('bin')))

# Import custom functions
from customPythonFunctions import generate_split_dir


#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300

overwriteFigures = True
overwriteData = True

**Loading annotation pkl**

In [2]:
step = "Step2"
next_step = "Step3"

In [3]:
annotation_col_name = 'annotation_step_2'

In [12]:
file_paths = glob.glob(str(here(f"02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/{step}/*/results/SCGT00_cluster_res*_annotated.pkl")))
# Load and concatenate all files into a single column DataFrame

In [13]:
for file_path in file_paths: 
    clusterAnnotDF = pd.read_pickle(file_path)
    print(set(clusterAnnotDF[annotation_col_name]))

{'T_CD8_NonNaive', 'LowQuality_cells', 'T_CD4_NonNaive'}
{'ILC', 'T_CD8_NonNaive', 'Platelets', 'LowQuality_cells'}
{'Doublets_Step2', 'DC'}
{'Mono', 'LowQuality_cells'}
{'T_CD4_Naive', 'T_CD4_NonNaive', 'Platelets', 'T_CD8_Naive', 'LowQuality_cells'}
{'Platelets', 'LowQuality_cells', 'B'}


In [14]:
clusterAnnotDF = pd.concat([pd.read_pickle(file_path) for file_path in file_paths], axis=0)

In [15]:
file_paths

['/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/Step2/T_NonNaive/results/SCGT00_cluster_res0.5_annotated.pkl',
 '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/Step2/NK/results/SCGT00_cluster_res0.5_annotated.pkl',
 '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/Step2/DC/results/SCGT00_cluster_res0.1_annotated.pkl',
 '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/Step2/Mono/results/SCGT00_cluster_res0.5_annotated.pkl',
 '/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellT

In [16]:
print(clusterAnnotDF.head())

                                       annotation_step_2
cellID                                                  
SCGT00_L051_I52.3P_T0_AAACCCACAACTGTGT  LowQuality_cells
SCGT00_L051_I57.3P_T0_AAACCCAGTCAAAGAT    T_CD4_NonNaive
SCGT00_L051_I55.3P_T0_AAACCCATCCCAATAG    T_CD8_NonNaive
SCGT00_L051_I52.3P_T0_AAACGAAAGGCTAACG    T_CD4_NonNaive
SCGT00_L051_I55.3P_T0_AAACGAAAGGTCCAGA    T_CD8_NonNaive


In [17]:
set(clusterAnnotDF[annotation_col_name])

{'B',
 'DC',
 'Doublets_Step2',
 'ILC',
 'LowQuality_cells',
 'Mono',
 'Platelets',
 'T_CD4_Naive',
 'T_CD4_NonNaive',
 'T_CD8_Naive',
 'T_CD8_NonNaive'}

In [18]:
print(clusterAnnotDF.shape)

(790342, 1)


In [19]:
clusterAnnotDF.to_pickle(here(f"02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/{step}/{step}_clusterAnnotDF.pkl"))

**Updating master annotation DF**

In [20]:
master_AnnotDFpath = str(here("02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/master_clusterAnnotDF.pkl"))
master_AnnotDF = pd.read_pickle(master_AnnotDFpath)
master_AnnotDF.head()

,annotation_step_1
cellID,
SCGT00_L051_I53.3P_T0_AACCAACGTGGGATTG,pDC
SCGT00_L051_I51.3P_T0_AAGGTAATCTTCCCGA,pDC
SCGT00_L051_I58.3P_T0_ACCTGAAAGTGCCAGA,pDC
SCGT00_L051_I51.3P_T0_ACGGGTCTCTTGGTGA,pDC
SCGT00_L051_I53.3P_T0_ACTATGGAGGCGCTCT,pDC


In [21]:
last_column_name = master_AnnotDF.columns[-1]
master_AnnotDF[annotation_col_name] = master_AnnotDF[last_column_name]
master_AnnotDF[annotation_col_name].update(clusterAnnotDF[annotation_col_name])

In [22]:
print(master_AnnotDF.head())

                                       annotation_step_1 annotation_step_2
cellID                                                                    
SCGT00_L051_I53.3P_T0_AACCAACGTGGGATTG               pDC               pDC
SCGT00_L051_I51.3P_T0_AAGGTAATCTTCCCGA               pDC               pDC
SCGT00_L051_I58.3P_T0_ACCTGAAAGTGCCAGA               pDC               pDC
SCGT00_L051_I51.3P_T0_ACGGGTCTCTTGGTGA               pDC               pDC
SCGT00_L051_I53.3P_T0_ACTATGGAGGCGCTCT               pDC               pDC


In [23]:
print(master_AnnotDF.shape)

(855417, 2)


In [24]:
set(master_AnnotDF[annotation_col_name])

{'B',
 'Cycling',
 'DC',
 'Doublets_Step1',
 'Doublets_Step2',
 'ILC',
 'LowQuality_cells',
 'Mono',
 'Plasma',
 'Platelets',
 'Progenitors',
 'RBC',
 'T_CD4_Naive',
 'T_CD4_NonNaive',
 'T_CD8_Naive',
 'T_CD8_NonNaive',
 'pDC'}

In [25]:
set(clusterAnnotDF[annotation_col_name])

{'B',
 'DC',
 'Doublets_Step2',
 'ILC',
 'LowQuality_cells',
 'Mono',
 'Platelets',
 'T_CD4_Naive',
 'T_CD4_NonNaive',
 'T_CD8_Naive',
 'T_CD8_NonNaive'}

In [26]:
# For step1
master_AnnotDF.to_pickle(here(f"02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/master_clusterAnnotDF.pkl"))

# Split Lineages

In [32]:
annotation_col_name = 'annotation_step_2'
not_interesting_categories = ["LowQuality_cells", "Doublets_Step1", "Doublets_Step2"]
finished_categories = ["Progenitors", "pDC", "Plasma", "RBC", "Platelets", "Cycling", "B", "DC", "Mono"]

In [33]:
cell_counts = master_AnnotDF.groupby(annotation_col_name).size().sort_values(ascending=False)
print(cell_counts)

annotation_step_2
T_CD4_NonNaive      160017
Mono                131392
T_CD8_NonNaive      130019
T_CD4_Naive         126306
ILC                  57046
B                    55869
Platelets            49789
LowQuality_cells     46475
T_CD8_Naive          40745
Doublets_Step1       34797
DC                   10440
Doublets_Step2        3361
pDC                   3125
Cycling               2741
Progenitors           1716
Plasma                1383
RBC                    196
dtype: int64


In [34]:
total_cells_to_remove = cell_counts.loc[not_interesting_categories].sum()
print(f"We are removing a total of {total_cells_to_remove} cells")
for category in not_interesting_categories:
    cells_removed = cell_counts.loc[category]
    print(f"{category}: {cells_removed} cells removed")

We are removing a total of 84633 cells
LowQuality_cells: 46475 cells removed
Doublets_Step1: 34797 cells removed
Doublets_Step2: 3361 cells removed


In [35]:
total_cells_finished = cell_counts.loc[finished_categories].sum()
print(f"We have finished a total of {total_cells_finished} cells")
for category in finished_categories:
    cells_finished = cell_counts.loc[category]
    print(f"{category}: {cells_finished} cells finished")

We have finished a total of 256651 cells
Progenitors: 1716 cells finished
pDC: 3125 cells finished
Plasma: 1383 cells finished
RBC: 196 cells finished
Platelets: 49789 cells finished
Cycling: 2741 cells finished
B: 55869 cells finished
DC: 10440 cells finished
Mono: 131392 cells finished


In [36]:
next_step_categories = cell_counts.index.difference(set(finished_categories).union(set(not_interesting_categories)))
total_cells_of_interest = cell_counts.loc[next_step_categories].sum()
print(f"We are still missing a total of {total_cells_of_interest} cells")
for category in next_step_categories:
    cells_missing = cell_counts.loc[category]
    print(f"{category}: {cells_missing} cells missing")

We are still missing a total of 514133 cells
ILC: 57046 cells missing
T_CD4_Naive: 126306 cells missing
T_CD4_NonNaive: 160017 cells missing
T_CD8_Naive: 40745 cells missing
T_CD8_NonNaive: 130019 cells missing


In [37]:
master_AnnotDF[annotation_col_name].unique()

array(['pDC', 'LowQuality_cells', 'Doublets_Step1', 'Platelets', 'Mono',
       'DC', 'Doublets_Step2', 'ILC', 'T_CD4_Naive', 'T_CD4_NonNaive',
       'T_CD8_Naive', 'T_CD8_NonNaive', 'Cycling', 'Plasma',
       'Progenitors', 'RBC', 'B'], dtype=object)

In [38]:
template_path = here('02_cell_annotation/SCGT00_CentralizedDataset/01_fromDataset_to_CellLineages/template')
template_notebook_list = [str(template_path) + '/' + f for f in os.listdir(template_path) if f.endswith('.ipynb')]

annotation_path = str(here(f"02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/{step}/{step}_clusterAnnotDF.pkl"))
targetdir = str(here(f'02_cell_annotation/SCGT00_CentralizedDataset/02_fromCellLineages_to_CellTypes/{next_step}'))
os.makedirs(targetdir)

for cellGroup in master_AnnotDF[annotation_col_name].unique():
    if cellGroup in next_step_categories: 
        print(cellGroup)
        # Generate the directory tree
        os.makedirs(here('{}/{}/results/figures'.format(targetdir, cellGroup)), exist_ok=False)
        for template_notebook in template_notebook_list:
            generate_split_dir(cellGroup = cellGroup, 
                               annotationPath = annotation_path, 
                               targetDir = targetdir, 
                               annotation_col_name = annotation_col_name, # Davide changed this line after editing the function. Check if still works
                               templateNotebook = template_notebook)
    else: 
        print(f"Skipping {cellGroup}")

Skipping pDC
Skipping LowQuality_cells
Skipping Doublets_Step1
Skipping Platelets
Skipping Mono
Skipping DC
Skipping Doublets_Step2
ILC
01_qualityControl_HVGselection.ipynb: cellGroup changed!
01_qualityControl_HVGselection.ipynb: annotationDFpath changed!
01_qualityControl_HVGselection.ipynb: annotation column changed!
03_compute_neighborsUMAP_QCmetrics.ipynb: cellGroup changed!
03_compute_neighborsUMAP_QCmetrics.ipynb: annotationDFpath changed!
03_compute_neighborsUMAP_QCmetrics.ipynb: annotation column changed!
05_annotation.ipynb: cellGroup changed!
02_integration_scVI.ipynb: cellGroup changed!
06_computeMarkers_specificResolution.ipynb: cellGroup changed!
04_clustering.ipynb: cellGroup changed!
T_CD4_Naive
01_qualityControl_HVGselection.ipynb: cellGroup changed!
01_qualityControl_HVGselection.ipynb: annotationDFpath changed!
01_qualityControl_HVGselection.ipynb: annotation column changed!
03_compute_neighborsUMAP_QCmetrics.ipynb: cellGroup changed!
03_compute_neighborsUMAP_QCmetri

In [43]:
session_info.show()